# **Intalação de Pacotes e Bibliotecas Necesárias**

In [ ]:
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install opencv-python matplotlib
!pip install 'git+https://github.com/facebookresearch/segment-anything.git'

# **Importação de Bibliotecas**

In [ ]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics import jaccard_score, precision_recall_fscore_support, accuracy_score
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# **Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Carregamento e Pré-processamento de Imagens**

In [ ]:
image = cv2.imread('/content/drive/MyDrive/Modelos/Dataset/test_images/da111ca5-520f-4ea8-97ad-ae5fce58ee91.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (512, 512))

In [ ]:
plt.figure(figsize=(7,7))
plt.imshow(image)
plt.axis('off')
plt.show()

# **Carregamento do Modelo SAM**

In [ ]:
sam_checkpoint = "/content/drive/MyDrive/Modelos/SAM/sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

# **Gerar Máscaras**

In [ ]:
mask_generator_ = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.9,
    stability_score_thresh=0.96,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing
)

In [ ]:
masks = mask_generator_.generate(image)
print(len(masks))

# **Visualização das Máscaras Geradas**

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.35)))


In [ ]:
plt.figure(figsize=(7,7))
plt.imshow(image)
show_anns(masks)
plt.axis('off')
plt.show()

# **Avaliação de Resultados**

In [ ]:
# Load the ground truth binary mask
ground_truth_mask = cv2.imread('/content/drive/MyDrive/Modelos/Dataset/test_masks/da111ca5-520f-4ea8-97ad-ae5fce58ee91.png', cv2.IMREAD_GRAYSCALE)
ground_truth_mask = cv2.resize(ground_truth_mask, (720, 720))
ground_truth_mask = (ground_truth_mask > 0).astype(np.uint8)  # Convert to binary

In [ ]:
# Generate the predicted binary mask
predicted_mask = np.zeros((512, 512), dtype=np.uint8)
for mask in masks:
    predicted_mask[mask['segmentation']] = 1

In [ ]:
# Flatten masks for metric calculations
ground_truth_flat = ground_truth_mask.flatten()
predicted_flat = predicted_mask.flatten()

In [ ]:
iou = jaccard_score(ground_truth_flat, predicted_flat)
print(f"IoU: {iou}")

In [ ]:
# Visualize the ground truth and predicted masks
plt.figure(figsize=(15, 7))
plt.subplot(1, 3, 1)
plt.imshow(image)
plt.title('Original Image')

plt.subplot(1, 3, 2)
plt.imshow(predicted_mask, cmap='gray')
plt.title('Predicted Mask')

plt.subplot(1, 3, 3)
plt.imshow(ground_truth_mask, cmap='gray')
plt.title('Ground Truth Mask')

plt.show()